In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("database/reporting.db")
log_df = pd.read_sql_query("SELECT * FROM upload_log WHERE table_alias = 'c0_budgetary_execution_details'", conn)
log_df

In [16]:
import sqlite3
import json
import os
import logging
import pandas as pd
from datetime import datetime
from io import BytesIO
from great_tables import GT, loc, style, px
from great_tables.data import islands

# Parameters
db_path = "database/reporting.db"
cutoff = pd.to_datetime("2025-04-15")
current_year = 2025
report = "Quarterly_Report"

# Configure logging
logging.basicConfig(level=logging.DEBUG)


# Fetch latest table data
def fetch_latest_table_data(conn: sqlite3.Connection, table_alias: str, cutoff: pd.Timestamp) -> pd.DataFrame:
    cutoff_str = cutoff.isoformat()
    query = """
        SELECT uploaded_at, id
        FROM upload_log
        WHERE table_alias = ?
        ORDER BY ABS(strftime('%s', uploaded_at) - strftime('%s', ?))
        LIMIT 1
    """
    result = conn.execute(query, (table_alias, cutoff_str)).fetchone()
    if not result:
        raise ValueError(f"No uploads found for table alias '{table_alias}' near cutoff {cutoff_str}")
    closest_uploaded_at, upload_id = result
    df = pd.read_sql_query(
        f"SELECT * FROM {table_alias} WHERE upload_id = ?",
        conn,
        params=(upload_id,)
    )
    return df

# Build commitment summary table
def build_commitment_summary_table(df: pd.DataFrame, current_year: int, report: str, db_path: str) -> pd.DataFrame:
    df = df[df["Budget Period"] == current_year]
    df = df[df["Fund Source"].isin(["VOBU", "EFTA"])]
    df["Programme"] = df["Functional Area Desc"].replace({
        "HORIZONEU_21_27": "HE",
        "H2020_14_20": "H2020"
    })
    agg = df.groupby("Programme")[
        ["Commitment Appropriation", "Committed Amount", "Commitment Available "]
    ].sum().reset_index()
    agg["%"] = agg["Committed Amount"] / agg["Commitment Appropriation"]
    agg = agg.rename(columns={
        "Commitment Appropriation": "Available Commitment Appropriations (1)",
        "Committed Amount": "L1 Commitment (2)",
        "Commitment Available ": "RAL on Appropriation (3)=(1)-(2)",
        "%": "% consumed of L1 and L2 against Commitment Appropriations (4) = (2)/(1)"
    })
    agg = agg.loc[agg["Programme"] == "HE"]

    BLUE        = "#004A99"
    LIGHT_BLUE  = "#e6eff9"
    GRID_CLR    = "#004A99"

    tbl = (
        GT(agg)
        .tab_header("Commitment Appropriations", subtitle="General Overview")
        .tab_stub(rowname_col="Programme")
        .tab_stubhead("Programme")
        # ── formats ────────────────────────────────────────────────────────────
        .fmt_number(columns=[
            "Available Commitment Appropriations (1)",
            "L1 Commitment (2)",
            "RAL on Appropriation (3)=(1)-(2)"
        ], accounting=True, decimals=2)
        .fmt_percent(
            columns="% consumed of L1 and L2 against Commitment Appropriations (4) = (2)/(1)",
            decimals=2
        )
        # ── Arial everywhere ──────────────────────────────────────────────────
        .opt_table_font(font="Arial")
        # ── HEADER + STUB COLOUR ──────────────────────────────────────────────
        .tab_style(
                        style = [
                             style.fill(color=BLUE),         # ← use “+”, not “&”
                             style.text(color="white", weight="bold")
                        ],
                        locations = loc.column_labels()
                  ) 
        .tab_style(
                    style = [
                        style.fill(color=BLUE),
                        style.text(color="white", weight="bold"),
                    ],
                    locations = loc.stub()
                   )
        
        # ── GRID LINES ────────────────────────────────────────────────────────
        .tab_style(
            style = style.borders(color=GRID_CLR, weight='0.5px'),
            locations = loc.body()
        )
        .tab_style(
            style = style.borders(color=GRID_CLR, weight='2px'),
            locations = loc.body()
        )
        # ── ROW STRIPING ───────────────────────────────────────────────────────
        .tab_options(row_striping_include_table_body=True, row_striping_background_color=LIGHT_BLUE)
        # ── SOURCE NOTE ────────────────────────────────────────────────────────
        .tab_source_note("Source: Summa DataWarehouse")
        .tab_source_note("Table_1a")
    )


    insert_variable(
        report=report,
        module="BudgetModule",
        var="table_1a_commitment_summary",
        value=agg.to_dict(orient="records"),
        db_path=db_path,
        anchor="table_1a",
        gt_table=tbl
    )
    return agg

# Insert variable
def insert_variable(report, module, var, value, db_path, anchor=None, gt_table=None):
    try:
        con = sqlite3.connect(db_path)
        logging.debug(f"Inserting variable: report={report}, module={module}, var={var}, anchor={anchor}, db_path={db_path}")
        print(f"Attempting to insert: {var}, value={value[:50]}...")  # Debug
        serialized_value = json.dumps(value, default=str)
        gt_image = None
        if gt_table:
            temp_file = f"temp_{var}.png"
            print(f"Saving GT table to {temp_file}")  # Debug
            gt_table.save(temp_file)
            with open(temp_file, "rb") as f:
                gt_image = f.read()
            os.remove(temp_file)
            print(f"Read PNG as binary, size={len(gt_image)} bytes")  # Debug
        con.execute('''
            INSERT INTO report_variables (report_name, module_name, var_name, anchor_name, value, gt_image, created_at)
            VALUES (?, ?, ?, ?, ?, ?, CURRENT_TIMESTAMP)
        ''', (report, module, var, anchor or var, serialized_value, gt_image))
        con.commit()
        logging.debug(f"Successfully inserted variable: {var}")
        print(f"Inserted variable: {var}")  # Debug
    except Exception as e:
        logging.error(f"Failed to insert variable {var}: {str(e)}")
        print(f"Error inserting {var}: {str(e)}")  # Debug
        raise
    finally:
        con.close()

# Main execution
try:
    conn = sqlite3.connect(db_path)
    df_comm = fetch_latest_table_data(conn, "c0_budgetary_execution_details", cutoff)
    table = build_commitment_summary_table(df_comm, current_year, report, db_path)
except Exception as e:
    print("Error:", e)
    logging.error(f"Main execution error: {str(e)}")
finally:
    conn.close()

DEBUG:root:Inserting variable: report=Quarterly_Report, module=BudgetModule, var=table_1a_commitment_summary, anchor=table_1a, db_path=database/reporting.db
DEBUG:selenium.webdriver.common.selenium_manager:Selenium Manager binary found at: c:\Users\vinci\anaconda3\envs\qenv\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe
DEBUG:selenium.webdriver.common.selenium_manager:Executing process: c:\Users\vinci\anaconda3\envs\qenv\Lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --debug --language-binding python --output json


Attempting to insert: table_1a_commitment_summary, value=[{'Programme': 'HE', 'Available Commitment Appropriations (1)': 2218254939.0, 'L1 Commitment (2)': 17040224.92, 'RAL on Appropriation (3)=(1)-(2)': 2201214714.08, '% consumed of L1 and L2 against Commitment Appropriations (4) = (2)/(1)': 0.007681815385783303}]...
Saving GT table to temp_table_1a_commitment_summary.png


DEBUG:selenium.webdriver.common.selenium_manager:Sending stats to Plausible: Props { browser: "chrome", browser_version: "", os: "windows", arch: "amd64", lang: "python", selenium_version: "4.32" }
DEBUG:selenium.webdriver.common.selenium_manager:chromedriver not found in PATH
DEBUG:selenium.webdriver.common.selenium_manager:chrome detected at C:\Program Files\Google\Chrome\Application\chrome.exe
DEBUG:selenium.webdriver.common.selenium_manager:Running command: wmic datafile where name='C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe' get Version /value
DEBUG:selenium.webdriver.common.selenium_manager:Output: "\r\r\n\r\r\nVersion=136.0.7103.49\r\r\n\r\r\n\r\r\n\r"
DEBUG:selenium.webdriver.common.selenium_manager:Detected browser: chrome 136.0.7103.49
DEBUG:selenium.webdriver.common.selenium_manager:Discovering versions from https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json
DEBUG:selenium.webdriver.common.selenium_manager:Required d

Read PNG as binary, size=17695 bytes
Inserted variable: table_1a_commitment_summary


In [ ]:
table

In [ ]:
import sqlite3
import logging
from io import BytesIO
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import ttk

# Configure logging
logging.basicConfig(level=logging.DEBUG)

# Parameters
db_path = "database/reporting.db"
var_name = "table_1a_commitment_summary"

# Fetch the image
def fetch_variable_image(db_path, var_name):
    conn = sqlite3.connect(db_path)
    try:
        cursor = conn.execute('''
            SELECT gt_image
            FROM report_variables
            WHERE var_name = ?
            ORDER BY created_at DESC
            LIMIT 1
        ''', (var_name,))
        result = cursor.fetchone()
        if result and result[0]:
            logging.debug(f"Fetched image for {var_name}, size={len(result[0])} bytes")
            return result[0]
        else:
            logging.warning(f"No image found for {var_name}")
            return None
    except sqlite3.Error as e:
        logging.error(f"Database error fetching image for {var_name}: {str(e)}")
        raise
    finally:
        conn.close()

# Fetch and display the image in a GUI
image_blob = fetch_variable_image(db_path, var_name)

if image_blob:
    # Convert BLOB to image
    try:
        image = Image.open(BytesIO(image_blob))
        logging.debug(f"Image dimensions: {image.size}")

        # Create GUI window
        window = tk.Tk()
        window.title("Loaded Table Image")
        window.geometry(f"{image.width}x{image.height}")

        # Convert PIL image to Tkinter-compatible image
        photo = ImageTk.PhotoImage(image)
        logging.debug("Converted image to PhotoImage")

        # Add image to label
        label = ttk.Label(window, image=photo)
        label.image = photo  # Keep a reference to avoid garbage collection
        label.pack()
        logging.debug("Packed label into window")

        # Optionally save to file for reference
        output_path = f"loaded_{var_name}.png"
        with open(output_path, "wb") as f:
            f.write(image_blob)
        logging.debug(f"Saved loaded image to {output_path}")
        print(f"Image displayed in GUI and saved as {output_path}. Verify the table formatting. Close the window to exit.")

        # Start the GUI loop
        window.mainloop()
    except Exception as e:
        logging.error(f"Error displaying image: {str(e)}")
        print(f"Error displaying image: {str(e)}. Check logs.")
else:
    print(f"No image found for {var_name}. Check the database or previous run.")